In [1]:
import sys
import os

# Get the path to the validate_assay folder
module_path = os.path.abspath(os.path.join('..', 'validate_assay'))

# Add the folder to the system path
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from pcrvalidationtools import *
from Bio import Entrez, SeqIO

In [3]:
# files 
INPUT_PATH = "../Data/v4_08262024/"
datasets_file = INPUT_PATH + 'enterovirus_metadata.tsv'
clustered_file =  INPUT_PATH + 'ev_clustered100.fasta'

# 1. Enterovirus 

## 1a. Filter

In [4]:
# filter for entered collection date, human host, completeness 
df_filter1 = filter_data(datasets_file)

print(f'There are {df_filter1.shape[0]} sequences after filtering for complete human-host sequences with a collection date')

There are 6495 sequences after filtering for complete human-host sequences with a collection date


In [6]:
# create a text file of one accessions per line to retrieve sequences for
accession_output_filename = "enterovirus_filtered_acc.txt"
df_filter1['Accession'].to_csv(INPUT_PATH + accession_output_filename, index=False, header=False)

print(f'Filtered accessions saved to {accession_output_filename}')

Filtered accessions saved to enterovirus_filtered_acc.txt


## 1b. Cluster at 100% identity with CD-HIT

In [7]:
# return list of accessions from fasta headers
# ex. header: >PP191126.1 Coxsackievirus A6 strain CVA6/16/GJ/KOR/2022, complete genome 
def get_acc_fasta(file_path):
    # parse headers
    headers = []
    with open(file_path, 'r') as fasta_file:
        for line in fasta_file:
            if line.startswith('>'):
                header = line.strip()[1:]
                headers.append(header)
    # retrieve accession from header
    accessions = [h.split(' ')[0] for h in headers]
    return accessions

In [8]:
# read in cd-hit output 
dereplicated_acc = get_acc_fasta(clustered_file)

# remove duplicate sequences 
df_filter2 = df_filter1[df_filter1['Accession'].isin(dereplicated_acc)].copy()

print(f'There are {df_filter2.shape[0]} sequences after dereplication')
print(f'{df_filter1.shape[0] - df_filter2.shape[0]} sequences were removed here')

There are 6149 sequences after dereplication
346 sequences were removed here


## 1c. Filter for genomes with 5' UTRs greater than 400 nt long

In [12]:
# use entrez to determine the 5' utr length 

# info for NCBI Entrez
Entrez.email = # email here
Entrez.api_key = # API key here 

# returns a list of queries
# each query is a string of comma separated accessions
# this breaks up the requests we send to NCBI
# accessions (list): list of accession number strings
def define_query(identifiers):
    # if getting invalid uid error, try lowering batch size
    BATCH_SIZE = 1000
    queries = []
    size = len(identifiers)
    iters = (size // BATCH_SIZE) + (size % BATCH_SIZE > 0)
    
    # make sure the list are strings
    identifiers = list(map(str, identifiers))
    
    for i in range(iters):
        start_range = i * BATCH_SIZE
        end_range = start_range + BATCH_SIZE
        query_str = ", ".join(identifiers[start_range:end_range])
        queries.append(query_str)
    return queries

# returns genbank records for the list of accessions
# accessions : list of accession strings
def get_gb_records(accessions):
    gb_records = []
    queries = define_query(accessions)
    for query in queries:
        handle = Entrez.efetch(db="nucleotide", id=query, rettype="gb", retmode="text")
        gb_records.extend(SeqIO.parse(handle, "genbank"))
        handle.close()
    return gb_records

# returns length of 5' utr 
# record: gb record
def get_utr_len(record):
    utr_len = np.nan
    # parse through features for CDS or 5'UTR
    for feature in record.features:
        if feature.type == '5\'UTR':
            # get 5' utr location
            loc = feature.location
            # define utr length as start of cds annotation
            utr_len = loc.start
            # if direction is reverse, calculate utr length
            if loc.strand == -1:
                utr_len = len(record.seq) - loc.start
            break
        if feature.type == 'CDS':
            # get cds location
            loc = feature.location
            # define utr length as start of cds annotation
            utr_len = loc.start
            # if direction is reverse, calculate utr length
            if loc.strand == -1:
                utr_len = len(record.seq) - loc.end
            break
    return utr_len

# returns a df with accession number and corresponding 5' utr length
def accession_utr_df(gb_records):
    # parse records and get utr length record
    df_dict = {'Accession': [], '5\'UTR Length': []}
    for record in gb_records:
        df_dict['Accession'].append(record.id)
        df_dict['5\'UTR Length'].append(get_utr_len(record))

    # return df
    return pd.DataFrame(df_dict)

In [13]:
# retrieve length of 5' utr for filtered genomes
gb_records = get_gb_records(df_filter2['Accession'].tolist())
utr_df = accession_utr_df(gb_records)

In [14]:
utr_df

,Accession,5'UTR Length
0,OR204681.1,694.0
1,OR204682.1,709.0
2,OR220808.1,572.0
3,OR220809.1,563.0
4,OR230416.1,710.0
...,...,...
6144,AB779614.1,0.0
6145,AB779615.1,0.0
6146,AB779616.1,0.0
6147,AB779617.1,0.0


In [15]:
# filter genomes for 5'UTR length >= 400 
valid_utr_acc = utr_df[utr_df['5\'UTR Length'] >= 400]['Accession'].tolist()
df_filter3 = df_filter2[df_filter2['Accession'].isin(valid_utr_acc)].copy()

print(f'There are {df_filter3.shape[0]} sequences after removing 5\' UTRs < 400nt')
print(f'{df_filter2.shape[0] - df_filter3.shape[0]} sequences were removed here')

There are 4676 sequences after removing 5' UTRs < 400nt
1473 sequences were removed here


In [16]:
# save filtered datasets as tsv file 
output_filename = 'ev_metadata_filtered.tsv'
df_filter3.to_csv(INPUT_PATH + output_filename, sep='\t', index=False)

print(f'Completely filtered genomes saved to {output_filename}')

Completely filtered genomes saved to ev_metadata_filtered.tsv


In [17]:
# check file saved 
ev_data = pd.read_csv(INPUT_PATH+output_filename, sep='\t') 
ev_data

,Accession,Virus Name,Virus Taxonomic ID,Host Taxonomic ID,Completeness,Isolate Collection date,Release date
0,OR204681.1,Coxsackievirus B3,12072,9606.0,COMPLETE,2021,2023-08-09T00:00:00Z
1,OR204682.1,Coxsackievirus B3,12072,9606.0,COMPLETE,2021,2023-08-09T00:00:00Z
2,OR220808.1,Rhinovirus C,463676,9606.0,COMPLETE,2023-02-23,2023-09-13T00:00:00Z
3,OR220809.1,Rhinovirus A,147711,9606.0,COMPLETE,2023-02-24,2023-09-13T00:00:00Z
4,OR230416.1,enterovirus D68,42789,9606.0,COMPLETE,2020,2023-07-24T00:00:00Z
...,...,...,...,...,...,...,...
4671,AB769160.1,Coxsackievirus A24,12089,9606.0,COMPLETE,2011-06,2012-12-05T00:00:00Z
4672,AB769162.2,Coxsackievirus A24,12089,9606.0,COMPLETE,2011-06,2012-12-05T00:00:00Z
4673,AB769163.1,Coxsackievirus A24,12089,9606.0,COMPLETE,2011-10,2012-12-05T00:00:00Z
4674,AB769164.2,Coxsackievirus A24,12089,9606.0,COMPLETE,2011-10,2012-12-05T00:00:00Z


## 1d. Species data

In [18]:
# returns genbank records for the list of accessions
# accessions : list of accession strings
def get_taxonomy_records(taxids):
    taxonomy_records = []
    queries = define_query(taxids)
    for query in queries:
        handle = Entrez.efetch(db="taxonomy", id=query, retmode="xml")
        records = Entrez.read(handle)
        handle.close()
        taxonomy_records.extend(records)
    return taxonomy_records

# returns a df with taxid and corresponding collapse taxid 
def taxonomy_collapse_df(records, target_taxid):
    # parse records and determine collapse taxid 
    df_dict = {'Virus Taxonomic ID': [], 'Collapse TaxId': [], 'Collapse Name': []}
    for record in records:
        
        taxid = int(record['TaxId'])
        collapse_taxid = None
        collapse_name = None
        
        # ordered from top of lineage to bottom (most specific)
        taxonomy_lineage = [node['TaxId'] for node in record['LineageEx']]
        
        # find the collapse taxid
        if str(target_taxid) in taxonomy_lineage:
            # the taxonomy level we want to collapse at is one level below the target 
            collapse_index = taxonomy_lineage.index(str(target_taxid)) + 1 
            # if index within range
            if collapse_index < len(taxonomy_lineage):
                collapse_taxid = taxonomy_lineage[collapse_index] 
                collapse_name = record['LineageEx'][collapse_index]['ScientificName']
            else:
                collapse_taxid = taxid 
                collapse_name = record['ScientificName']
                
        df_dict['Virus Taxonomic ID'].append(int(taxid))
        df_dict['Collapse TaxId'].append(int(collapse_taxid))
        df_dict['Collapse Name'].append(collapse_name)

    # return df
    return pd.DataFrame(df_dict)

In [19]:
ev_txid = 12059
taxids = set(ev_data['Virus Taxonomic ID'])
records = get_taxonomy_records(taxids)
taxonomy_collapse = taxonomy_collapse_df(records, ev_txid)
taxonomy_collapse

,Virus Taxonomic ID,Collapse TaxId,Collapse Name
0,1435148,138948,Enterovirus A
1,185889,147712,Rhinovirus B
2,185891,147711,Rhinovirus A
3,2094116,463676,Rhinovirus C
4,185893,147711,Rhinovirus A
...,...,...,...
166,35293,138949,Enterovirus B
167,35295,138949,Enterovirus B
168,35294,138949,Enterovirus B
169,297248,138948,Enterovirus A


In [21]:
# save as tsv file 
output_filename = 'ev_taxonomy_collapse.tsv'
taxonomy_collapse.to_csv(INPUT_PATH + output_filename, sep='\t', index=False)

print(f'Saved to {output_filename}')

Saved to ev_taxonomy_collapse.tsv
